In [1]:
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import sys

In [4]:
# Define image and batch size
img_height = 256
img_width = 256
batch_size = 64

# Load and Compile the model
model = tf.keras.models.load_model('model.h5')

sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.95, decay=0.0004)

model.compile(
    optimizer=sgd, 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

# Create the Generator
test_data_gen = ImageDataGenerator(preprocessing_function=None,
    rescale=1./255)

# Read the positive test samples.
test_generator = test_data_gen.flow_from_directory(
                  directory = r"./patches/test/",
                  target_size=(img_width, img_height), color_mode='grayscale',
                  batch_size=batch_size, class_mode="categorical")

Found 4875 images belonging to 3 classes.


In [5]:
print('\n# Evaluate on test data')
results = model.evaluate(test_generator)
print('test loss, test acc:', results)


# Evaluate on test data
77/77 [==============================] - 57s 740ms/step - loss: 0.2686 - accuracy: 0.9265
test loss, test acc: [0.26856313186225955, 0.9264958]


In [6]:
# Make predictions for the positive samples.
predictions = model.predict_generator(test_generator)
true_pos = 0
pos_size = len(predictions)

In [7]:
num = pos_size // batch_size

for i in range(num):
    label_batch = next(test_generator)[1]
    for j in range(batch_size):
        idx = np.argmax(predictions[i * batch_size + j])
        if (label_batch[j, idx] == 1):
            true_pos = true_pos + 1

accuracy = true_pos

print("Test set accuracy: {:.3%}".format(accuracy))

Test set accuracy: 198500.000%
